In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import urllib.request as REQ
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd
from tqdm import tqdm

u = "https://www.klac.or.kr/legalinfo/counselView.do?pageIndex=1&folderId=006&caseId=case-"
sentList1 = [] #제목 리스트(질문이 짧게 요약된 형태, 제목도 bert 학습 시 사용)
sentList2 = [] #질문 리스트
labelList = [] #레이블 리스트 손해배상이면 0, 민사일반이면 1 … 상속이면 6
numList = []#case(민법, 친족, 등등)
sentList3 = []
for i in range(1, 22):
  s = str(i)
  if len(s) ==2:
    s = '0'+s
  else:
    s = '00'+s
  numList.append(s)
print(numList)
df = pd.DataFrame(columns=['title', 'question', 'answer', 'label'])
for n in tqdm(range(len(numList))):
  i = 1
  while True:
    num = str(i)
    for k in range(5-len(num)):
      num = '0' + num
    url = u + numList[n] + '-' + num
    #print(url)
    try:
      html = REQ.urlopen(url).read()
      soup = BeautifulSoup(html, "html.parser")
      try:
        wList1 = soup.select('dd')
        if len(wList1[4].text) == 0:
          break
        sentList1.append(wList1[4].text) #제목 추가
        sentList2.append(wList1[5].text) #질문 추가
        sentList3.append(wList1[6].text) #답변 추가
        labelList.append(n)
      except:
        print('오류')
    except:
      print('오류')
    i += 1
qDic = {}
qDic['title'] = sentList1
qDic['question'] = sentList2
qDic['answer'] = [s.split('\n\n※ 주의')[0] for s in sentList3]
qDic['label'] = labelList
df = pd.DataFrame(qDic, columns = ['title', 'question', 'answer', 'label'])
df.index = range(len(df))

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021']


100%|██████████| 21/21 [2:13:24<00:00, 381.16s/it]


In [ ]:
df.head()

,title,question,answer,label
0,최저임금제도,"최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?",최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는...,0
1,퇴직금지급규정이 적용되는 사업장의 범위,저는 상시 근로자수 13인인 甲회사에서 1983년 10월부터 근무하다가 1개월 전 ...,퇴직금제도에 관하여 「근로기준법」제34조에 의하면 「근로자퇴직급여 보장법」이 정하는...,0
2,회사분할로 퇴직금 일괄수령 후 재입사 형식 취한 경우 계속근로인지,저는 甲회사에 입사하여 5년째 되던 해 소속부서 업무가 乙회사로 독립되자 甲회사에서...,기업의 합병·분할·영업양도 등의 경우 근로자들이 조직변경 전후에 계속하여 근무를 하...,0
3,학습지판매위탁 상담교사도 근로기준법상의 근로자인지,저는 학습지제작·판매회사인 甲회사와 위탁업무계약을 체결한 교육상담교사입니다. 저와 ...,「근로기준법」상 근로자의 범위에 관하여 판례는 “근로기준법상의 근로자에 해당하는지 ...,0
4,대학병원의 전공의가 근로기준법상의 근로자에 해당하는지,저는 甲대학병원에서 전공의 과정을 밟고 있는 레지던트입니다. 그런데 몇 달 전부터 ...,「근로기준법」제2조 제1항 1호에서는 ‘근로자’를 ‘직업의 종류와 관계없이 임금을 ...,0


In [ ]:
from tqdm import tqdm
question = list(df['question'])
question.extend(df['title'])
answer = list(df['answer'])
answer.extend(df['answer'])
new = {'question':question, 'answer':answer}
new = pd.DataFrame(new)
new.to_csv('/content/drive/MyDrive/파일위치/gujo_qna.csv')

In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 30.3 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Kkma

kkma = Kkma()

In [ ]:
import re
stopwords =['하', '의', '에', '이', '는', '을', '가', '되', '를', '있', '여', '으로', '경우', '는가', '수', '아', '로', '그', '은', '고', '제', '것', '지', '에서', '저', '대하', '조', '어', '등', '행위', '관하', '해당', '보', '항', '에게', '및', '과', '받', '와', '없', '의하', '다고', '처분', '않', '또는', '였', '었', '나요', '따르']
def tokenize_sentence(sentence):
  tokenized_sentence = kkma.morphs(sentence) # 토큰화
  stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
  #print(tok[:5])
  result = ''
  for s in stopwords_removed_sentence:
        s = re.sub(r"[^가-힣\s]", " ", s)
        #s = re.sub("\s\s+", " ", s)
        result += s +' '
  return result

In [ ]:
from tqdm import tqdm
question = [i]
for i in tqdm(range(len(new))):
  question[i] = tokenize_sentence(new['question'][i])


new['question'] = question
new.to_pickle('/content/drive/MyDrive/파일위치/gujo_qna_kkm.pkl')

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
emd = []
for q in tqdm(new['question']):
    emd.append(model.encode(q))
new['embedding'] = emd
new.to_csv('/content/drive/MyDrive/파일위치/gujo_qna_kkm_sbert.csv')